In [ ]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
text_df = pd.read_csv("/content/fake_or_real_news.csv")
text = list(text_df.text.values)
joined_text = " ".join(text)

In [ ]:
partial_text = joined_text[:100000]

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
unique_tokens= np.unique(tokens)
unique_token_index = { token: idx for idx, token in enumerate(unique_tokens) }

In [ ]:
n_words= 10
input_words =[]
next_words = []
for i in range(len(tokens)-n_words):
  input_words.append(tokens[i:i+n_words])
  next_words.append(tokens[i+n_words])

In [ ]:
x= np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y= np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [ ]:
for i, words in enumerate(input_words):
  for j, word in enumerate(words):
    x[i, j, unique_token_index[word]]=1
  y[i, unique_token_index[next_words[i]]]=1

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(x, y, batch_size=128, epochs=10, shuffle=True)

Epoch 1/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 60s 410ms/step - accuracy: 0.1100 - loss: 5.7358
Epoch 2/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 81s 404ms/step - accuracy: 0.1302 - loss: 5.4193
Epoch 3/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 82s 406ms/step - accuracy: 0.1616 - loss: 5.0844
Epoch 4/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 55s 405ms/step - accuracy: 0.1753 - loss: 4.8393
Epoch 5/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 58s 428ms/step - accuracy: 0.2140 - loss: 4.4906
Epoch 6/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 79s 406ms/step - accuracy: 0.2529 - loss: 4.1629
Epoch 7/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 81s 401ms/step - accuracy: 0.2927 - loss: 3.8491
Epoch 8/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 53s 389ms/step - accuracy: 0.3448 - loss: 3.4997
Epoch 9/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 83s 400ms/step - accuracy: 0.3868 - loss: 3.2296
Epoch 10/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 82s 399ms/step - accuracy: 0.4374 - loss: 2.9197


In [ ]:
model.save("mymodel.h5")
model= load_model("mymodel.h5")

In [ ]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [ ]:
possible = predict_next_word("The president of the united states has announced that he", 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step


In [ ]:
print([unique_tokens[idx] for idx in possible])

[np.str_('doesn'), np.str_('wants'), np.str_('are'), np.str_('will'), np.str_('was')]


In [ ]:
def generate_text(input_text, text_length, creativity=3):
  word_sequence = input_text.split(' ')
  current = 0
  for _ in range(text_length):
    sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
    try:
      choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
    except:
      choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current +=1
  return " ".join(word_sequence)

In [ ]:
generate_text("The president of the united states has announced that he", 100, 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 

'The president of the united states has announced that he doesn t show this election it was on new evidence but it are gone a very race in the house clinton is in women to get democrats who had clear elsewhere to get people until a legislation himself though but also was lead on ernst there was and new new same thing a bernie resolution with them there is about that day that tend and just at as women as bill the same news majority at a man party who was out of tehran until and what they less say when get more now no an school r in iowa'